In [2]:
import numpy as np
import pandas as pd
import scipy.io
import h5py

# Data Import

In [3]:
v1 = scipy.io.loadmat('data/monkey_spikes/SPKcounts_V20160929cue_MW_250X250ms.mat')
v2 = scipy.io.loadmat('data/monkey_spikes/SPKcounts_V20160930cue_MW_250X250ms.mat')
v3 = scipy.io.loadmat('data/monkey_spikes/SPKcounts_V20161005cue_MW_250X250ms.mat')
v4 = scipy.io.loadmat('data/monkey_spikes/SPKcounts_V20161017cue_MW_250X250ms.mat')
w1 = scipy.io.loadmat('data/monkey_spikes/SPKcounts_W20160112cue_MW_250X250ms.mat')
w2 = scipy.io.loadmat('data/monkey_spikes/SPKcounts_W20160113cue_MW_250X250ms.mat')
w3 = scipy.io.loadmat('data/monkey_spikes/SPKcounts_W20160121cue_MW_250X250ms.mat')
w4 = scipy.io.loadmat('data/monkey_spikes/SPKcounts_W20160122cue_MW_250X250ms.mat')
vs = [v1, v2, v3, v3]
ws = [w1, w2, w3, w4]

v1 = h5py.File('data/monkey_spikes/RastVect-V20160929-binsize10ms-align2StimOnset.mat')
v2 = h5py.File('data/monkey_spikes/RastVect-V20160930-binsize10ms-align2StimOnset.mat')
v3 = h5py.File('data/monkey_spikes/RastVect-V20161005-binsize10ms-align2StimOnset.mat')
v4 = h5py.File('data/monkey_spikes/RastVect-V20161017-binsize10ms-align2StimOnset.mat')
w1 = h5py.File('data/monkey_spikes/RastVect-W20160112-binsize10ms-align2StimOnset.mat')
w2 = h5py.File('data/monkey_spikes/RastVect-W20160113-binsize10ms-align2StimOnset.mat')
w3 = h5py.File('data/monkey_spikes/RastVect-W20160121-binsize10ms-align2StimOnset.mat')
w4 = h5py.File('data/monkey_spikes/RastVect-W20160122-binsize10ms-align2StimOnset.mat')
vs2 = [v1, v2, v3, v4]
ws2 = [w1, w2, w3, w4]

v1 = h5py.File('data/monkey_spikes/RastVect-V20160929-binsize10ms-align2SaccadeOnset.mat')
v2 = h5py.File('data/monkey_spikes/RastVect-V20160930-binsize10ms-align2SaccadeOnset.mat')
v3 = h5py.File('data/monkey_spikes/RastVect-V20161005-binsize10ms-align2SaccadeOnset.mat')
v4 = h5py.File('data/monkey_spikes/RastVect-V20161017-binsize10ms-align2SaccadeOnset.mat')
w1 = h5py.File('data/monkey_spikes/RastVect-W20160112-binsize10ms-align2SaccadeOnset.mat')
w2 = h5py.File('data/monkey_spikes/RastVect-W20160113-binsize10ms-align2SaccadeOnset.mat')
w3 = h5py.File('data/monkey_spikes/RastVect-W20160121-binsize10ms-align2SaccadeOnset.mat')
w4 = h5py.File('data/monkey_spikes/RastVect-W20160122-binsize10ms-align2SaccadeOnset.mat')
vs3 = [v1, v2, v3, v4]
ws3 = [w1, w2, w3, w4]

all_values = [vs, ws]
cue_aligned = [vs2, ws2]
choice_aligned = [vs3, ws3]

# Values

In [4]:
dfs = []

for monkey in ['V', 'W']:
    files = vs if monkey=='V' else ws
    for f, file in enumerate(files):
        print(f)
        df = pd.DataFrame()
        data = file['Y']
        bid = data[:,7]
        block = data[:,9]
        trial = data[:,5]
        letter = data[:,0]
        location = data[:,1]
        best_chosen = data[:,4]
        block_completed = data[:,12]
        rewards = data[:,2]
        idxs = np.argwhere(data[:,6]==1)[:,0]
        reversal_at_trial = {}
        for idx in idxs:
            bb = bid[idx]
            tt = trial[idx]
            reversal_at_trial[bb]=tt
        left = []
        right = []
        cor_loc = []
        cor_let = []
        for row in range(data.shape[0]):
            # 1. Determine letter mapping for left/right
            if location[row] == 0:  # Monkey chose Left
                l_let, r_let = ('A', 'B') if letter[row] == 0 else ('B', 'A')
            else:                   # Monkey chose Right
                l_let, r_let = ('B', 'A') if letter[row] == 0 else ('A', 'B')
            left.append(l_let)
            right.append(r_let)
            # 2. Determine Correct Location (cor_loc)
            # If best_chosen is 1, the chosen location was correct. 
            # If 0, the other location was correct.
            if best_chosen[row] == 1:
                cor_loc.append('left' if location[row] == 0 else 'right')
            else:
                cor_loc.append('right' if location[row] == 0 else 'left')
            # 3. Determine Correct Letter (cor_let)
            # If best_chosen is 1, the chosen letter was correct.
            # If 0, the other letter was correct.
            chosen_letter_code = letter[row] # 0 for A, 1 for B
            if best_chosen[row] == 1:
                cor_let.append('A' if chosen_letter_code == 0 else 'B')
            else:
                cor_let.append('B' if chosen_letter_code == 0 else 'A')
        # Assign to DataFrame
        df['block'] = bid
        df['block_type'] = block
        df['trial'] = trial
        df['left'] = left
        df['right'] = right
        df['cor_loc'] = cor_loc
        df['cor_let'] = cor_let
        df['clet'] = letter
        df['cloc'] = location
        df['monkey_accuracy'] = best_chosen
        df['reward'] = rewards
        df['block_completed'] = block_completed
        df = df[df['block_completed'] != 0]
        temp = []
        for bb in df['block'].unique():
            for t in df.query("block==@bb")['trial'].unique():
                temp.append(reversal_at_trial[bb])
        df['reversal_at_trial'] = temp
        df['monkey'] = [monkey for _ in range(df.shape[0])]
        df['session'] = [f for _ in range(df.shape[0])]
        dfs.append(df)

behavioral_data = pd.concat(dfs, ignore_index=True)
behavioral_data['block_type'] = behavioral_data['block_type'].replace(1, 'what')
behavioral_data['block_type'] = behavioral_data['block_type'].replace(2, 'where')
behavioral_data['cloc'] = behavioral_data['cloc'].replace(1, 'right')
behavioral_data['cloc'] = behavioral_data['cloc'].replace(0, 'left')
behavioral_data['clet'] = behavioral_data['clet'].replace(0, 'A')
behavioral_data['clet'] = behavioral_data['clet'].replace(1, 'B')
behavioral_data = behavioral_data.drop(columns='block_completed')

0
1
2
3
0
1
2
3


In [5]:
behavioral_data.to_pickle("data/empirical2.pkl")

In [13]:
def check_opposite(row):
    return (row['left'] == 'A' and row['right'] == 'B') or \
           (row['left'] == 'B' and row['right'] == 'A')

# Apply the check to all rows
all_opposite = behavioral_data.apply(check_opposite, axis=1).all()

print("All rows contain opposite letters:", all_opposite)

All rows contain opposite letters: True


# Spikes

In [3]:
def save_spikes_hdf5(spikes, filename):
    """Save spike dict spikes[trial] -> ndarrayinto HDF5 with gzip compression."""
    with h5py.File(filename, "w") as f:
        for trial, arr in spikes.items():
            f.create_dataset(str(trial), data=arr, compression="gzip", compression_opts=4, shuffle=True)

In [4]:
def convert_monkey_spikes_to_hdf5(all_values, cue_aligned, choice_aligned, align_to='choice'):

    for m, monkey in enumerate(['V', 'W']):
        files_value = all_values[m]
        files_cue = cue_aligned[m]
        files_choice = choice_aligned[m]

        for session in [0, 1, 2, 3]:
            print(f"\n=== Monkey {monkey}, session {session} ===")
            trial_data = files_value[session]['Y']
            bid = trial_data[:, 7]  # block ID
            tid = trial_data[:, 5]  # trial ID
            spk_cue_all = files_cue[session]['aligned2event']
            spk_choice_all = files_cue[session]['aligned2event']

            for block in np.unique(bid):
                block = int(block)
                print(f"  Processing block {block}...")
                spikes_block = {}
                block_trials = np.where(bid == block)[0]
                for idx in block_trials:
                    trial = int(tid[idx])
                    X_cue = spk_cue_all[idx]
                    X_choice = spk_choice_all[idx]
                    if align_to=='cue':
                        spikes_block[trial] = X_cue.T
                    elif align_to=='choice':
                        spikes_block[trial] = X_choice.T

                out_file = f"data/emp/monkey{monkey}_session{session}_block{block}_spikes.h5"
                print(f"    Saving: {out_file}")
                save_spikes_hdf5(spikes_block, out_file)

In [6]:
convert_monkey_spikes_to_hdf5(all_values, cue_aligned, choice_aligned, align_to='choice')


=== Monkey V, session 0 ===
  Processing block 1...
    Saving: data/emp/monkeyV_session0_block1_spikes.h5
  Processing block 2...
    Saving: data/emp/monkeyV_session0_block2_spikes.h5
  Processing block 3...
    Saving: data/emp/monkeyV_session0_block3_spikes.h5
  Processing block 4...
    Saving: data/emp/monkeyV_session0_block4_spikes.h5
  Processing block 5...
    Saving: data/emp/monkeyV_session0_block5_spikes.h5
  Processing block 6...
    Saving: data/emp/monkeyV_session0_block6_spikes.h5
  Processing block 7...
    Saving: data/emp/monkeyV_session0_block7_spikes.h5
  Processing block 8...
    Saving: data/emp/monkeyV_session0_block8_spikes.h5
  Processing block 9...
    Saving: data/emp/monkeyV_session0_block9_spikes.h5
  Processing block 10...
    Saving: data/emp/monkeyV_session0_block10_spikes.h5
  Processing block 11...
    Saving: data/emp/monkeyV_session0_block11_spikes.h5
  Processing block 12...
    Saving: data/emp/monkeyV_session0_block12_spikes.h5
  Processing bloc